In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D 
from keras.utils import np_utils
import pandas as pd
from keras import backend as K 
K.set_image_dim_ordering('th')

/home/akshaydv/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
seed = 7
np.random.seed(seed)

In [3]:
dataset = pd.read_csv('train.csv')
X = dataset.iloc[:, 1:].values.astype('float32')
y = dataset.iloc[:, 0:1].values

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
X_train = X_train / 255
X_test = X_test / 255
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [6]:
def CNN_Model():
    model = Sequential()
    model.add(Conv2D(30, (6, 6), input_shape=(1,28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
model = CNN_Model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=350)
scores = model.evaluate(X_test, y_test, verbose=0)

Train on 33600 samples, validate on 8400 samples
Epoch 1/25
33600/33600 [==============================] - 17s 513us/step - loss: 1.1635 - acc: 0.5998 - val_loss: 0.2411 - val_acc: 0.9313
Epoch 2/25
33600/33600 [==============================] - 17s 507us/step - loss: 0.3248 - acc: 0.8999 - val_loss: 0.1396 - val_acc: 0.9586
Epoch 3/25
33600/33600 [==============================] - 17s 503us/step - loss: 0.2114 - acc: 0.9354 - val_loss: 0.1047 - val_acc: 0.9681
Epoch 4/25
33600/33600 [==============================] - 17s 509us/step - loss: 0.1728 - acc: 0.9495 - val_loss: 0.0891 - val_acc: 0.9720
Epoch 5/25
33600/33600 [==============================] - 18s 547us/step - loss: 0.1463 - acc: 0.9561 - val_loss: 0.0804 - val_acc: 0.9762
Epoch 6/25
33600/33600 [==============================] - 18s 523us/step - loss: 0.1265 - acc: 0.9615 - val_loss: 0.0735 - val_acc: 0.9787
Epoch 7/25
33600/33600 [==============================] - 17s 521us/step - loss: 0.1147 - acc: 0.9665 - val_loss: 0.0

In [8]:
test_dataset = pd.read_csv('test.csv')
test_set = test_dataset.iloc[:,:].values.astype('float32')
test_set = test_set/255
test_set = test_set.reshape(test_set.shape[0], 1, 28, 28).astype('float32')
result = model.predict(test_set)

In [9]:
import csv
with open('result.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['ImageId'] + ['Label'])
    for i in range(len(result)):
        spamwriter.writerow([str(i+1)] + [result[i].argmax(axis=0)])                   